In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2


In [3]:
df = pd.read_csv('coffee_beans.csv')


In [4]:
def load_images(df):
    X = []
    y = []
    for _, row in df.iterrows():
        img = cv2.imread(row["filepaths"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = img / 255.0  # normalize
    
        X.append(img)
        y.append(row["class index"])
    
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.int32)
    return X, y



In [5]:
df_train = df[df["filepaths"].str.startswith("train/")]
df_test = df[df["filepaths"].str.startswith("test/")]

X_train, y_train = load_images(df_train)
X_test, y_test = load_images(df_test)


In [7]:
tf.keras.utils.set_random_seed(42) # seed for reproducibility

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential([
    Flatten(input_shape=(224, 224, 3)), # ??? have to research
    Dense(units=25, activation='relu'),
    Dense(units=15, activation='relu'),
    Dense(4, activation='linear'), # since the from_logits is used in the loss function
], name="CoffeeRoastAI")

In [8]:
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # gradient descent optimatiation
    loss= SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'], # ??? have to research
)

In [9]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10


2026-02-02 09:51:16.186909: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 722534400 exceeds 10% of free system memory.


38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.3292 - loss: 15.2463
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.5458 - loss: 3.1492
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.6533 - loss: 1.4685
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.6567 - loss: 1.9631
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.6308 - loss: 1.9099
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.7600 - loss: 1.0299
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.8125 - loss: 0.6008
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.7983 - loss: 0.6674
Epoch 9/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.8567 - loss: 0.4265
Epoch 10/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9117 - loss: 0.2260


In [8]:
model.summary()

Model: "CoffeeRoastAI"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 150528)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │     3,763,225 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │            64 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,291,039 (43.07 MB)

 Trainable params: 3,763,679 (14.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,527,360 (28.71 MB)

Because softmax is integrated into the output layer the output s a vector of probabilities.

In [9]:
logits = model(X_train) # outpus z_1, .., z_n and not a_1, .., a_n
f_x = tf.nn.softmax(logits) # map the z result to softmax function
f_x

<tf.Tensor: shape=(1200, 4), dtype=float32, numpy=
array([[9.9964762e-01, 1.1098160e-13, 1.6639702e-09, 3.5240385e-04],
       [9.9995989e-01, 2.0835024e-12, 4.2700049e-10, 4.0056661e-05],
       [9.9989218e-01, 5.1829987e-13, 1.1674608e-09, 1.0776618e-04],
       ...,
       [8.0938369e-01, 1.1781326e-06, 1.7837381e-04, 1.9043678e-01],
       [9.8214537e-01, 4.4037179e-08, 1.9885000e-04, 1.7655836e-02],
       [9.9740505e-01, 1.0036049e-08, 1.6469012e-07, 2.5948710e-03]],
      shape=(1200, 4), dtype=float32)>

In [10]:
class_names = {
    0: "Dark",
    1: "Green",
    2: "Light",
    3: "Medium"
}


In [11]:
IMG_SIZE = 224

img = cv2.imread("test/Medium/medium (13).png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
img = img / 255.0

img = np.expand_dims(img, axis=0)  # shape (1, 224, 224, 3)

pred = model.predict(img)
pred_class = np.argmax(pred, axis=1)[0]

print("Predicted:", class_names[pred_class])
print("Class probabilities (Dark, Green, Light, Medium):", pred[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Predicted: Medium
Class probabilities (Dark, Green, Light, Medium): [-29.29936  -45.98148  -40.205154 -26.681593]


Save the model

In [11]:
model.save("coffee_roast_model.keras")